In [1]:
import pandas as pd
from warnings import simplefilter
simplefilter(action='ignore', category = FutureWarning)
from warnings import simplefilter
simplefilter(action='ignore', category = FutureWarning)
%matplotlib inline

In [2]:
df = pd.read_csv('processed.cleveland.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [4]:
del df["trestbps"]
del df["chol"]
del df["age"]
del df["thalach"]
df

,sex,cp,fbs,restecg,exang,oldpeak,slope,ca,thal,target
0,1,1,1,2,0,2.3,3,0,6,0
1,1,4,0,2,1,1.5,2,3,3,2
2,1,4,0,2,1,2.6,2,2,7,1
3,1,3,0,0,0,3.5,3,0,3,0
4,0,2,0,2,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...
292,0,4,0,0,1,0.2,2,0,7,1
293,1,1,0,0,0,1.2,2,0,7,1
294,1,4,1,0,0,3.4,2,2,7,2
295,1,4,0,0,1,1.2,2,1,7,3


In [5]:

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
# df['sex'] = df.sex.map({0: 'female', 1: 'male'})

# df['thal'] = df.thal.fillna(df.thal.mean())
# df['ca'] = df.ca.fillna(df.ca.mean())

In [6]:
#########################################   Logistic Regression  #############################################################
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print('Accuracy for training set for Logistic Regression = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Logistic Regression = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))


Accuracy for training set for Logistic Regression = 0.8481012658227848
Accuracy for test set for Logistic Regression = 0.8333333333333334


In [7]:
classifier.coef_

array([[ 0.2655364 ,  0.24779751, -0.43803552,  0.13979613,  1.24959932,
         0.42972294, -0.05730478,  0.79514682,  0.30885241]])

In [8]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

In [9]:
def create_model():
    model=Sequential()
    model.add(Dense(6,activation="relu",input_dim=9))
    model.add(Dense(3,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

In [10]:
kfold=StratifiedKFold(5)
result=[]
m=0
for i,j in kfold.split(X,y):
    X_train=np.array([X[k] for k in i])
    y_train=np.array([y[k] for k in i])
    X_test=np.array([X[k] for k in j])
    y_test=np.array([y[k] for k in j])
    model=create_model()
    model.fit(X_train,y_train,epochs=1000)
    temp=model.evaluate(X_test,y_test,verbose=2)
    result.append(temp)
    if temp[1]>m:
        m=temp[1]
        model.save("best_nn.hdf5")

Train on 237 samples
Epoch 1/1000
237/237 [==============================] - 0s 2ms/sample - loss: 0.6860 - accuracy: 0.4937
Epoch 2/1000
237/237 [==============================] - 0s 64us/sample - loss: 0.6668 - accuracy: 0.5190
Epoch 3/1000
237/237 [==============================] - 0s 60us/sample - loss: 0.6523 - accuracy: 0.5570
Epoch 4/1000
237/237 [==============================] - 0s 68us/sample - loss: 0.6429 - accuracy: 0.6329
Epoch 5/1000
237/237 [==============================] - 0s 87us/sample - loss: 0.6346 - accuracy: 0.6793
Epoch 6/1000
237/237 [==============================] - 0s 95us/sample - loss: 0.6301 - accuracy: 0.6962
Epoch 7/1000
237/237 [==============================] - 0s 66us/sample - loss: 0.6250 - accuracy: 0.7089
Epoch 8/1000
237/237 [==============================] - 0s 68us/sample - loss: 0.6218 - accuracy: 0.7553
Epoch 9/1000
237/237 [==============================] - 0s 78us/sample - loss: 0.6184 - accuracy: 0.7595
Epoch 10/1000
237/237 [============

237/237 [==============================] - 0s 59us/sample - loss: 0.4660 - accuracy: 0.7848
Epoch 79/1000
237/237 [==============================] - 0s 58us/sample - loss: 0.4662 - accuracy: 0.7848
Epoch 80/1000
237/237 [==============================] - 0s 72us/sample - loss: 0.4647 - accuracy: 0.7890
Epoch 81/1000
237/237 [==============================] - 0s 59us/sample - loss: 0.4635 - accuracy: 0.7890
Epoch 82/1000
237/237 [==============================] - 0s 64us/sample - loss: 0.4627 - accuracy: 0.7848
Epoch 83/1000
237/237 [==============================] - 0s 66us/sample - loss: 0.4625 - accuracy: 0.7848
Epoch 84/1000
237/237 [==============================] - 0s 56us/sample - loss: 0.4612 - accuracy: 0.7890
Epoch 85/1000
237/237 [==============================] - 0s 59us/sample - loss: 0.4606 - accuracy: 0.7932
Epoch 86/1000
237/237 [==============================] - 0s 60us/sample - loss: 0.4603 - accuracy: 0.7848
Epoch 87/1000
237/237 [==============================] - 0s 

237/237 [==============================] - 0s 56us/sample - loss: 0.4243 - accuracy: 0.8017
Epoch 155/1000
237/237 [==============================] - 0s 62us/sample - loss: 0.4241 - accuracy: 0.8017
Epoch 156/1000
237/237 [==============================] - 0s 61us/sample - loss: 0.4235 - accuracy: 0.8017
Epoch 157/1000
237/237 [==============================] - 0s 51us/sample - loss: 0.4230 - accuracy: 0.8017
Epoch 158/1000
237/237 [==============================] - 0s 54us/sample - loss: 0.4229 - accuracy: 0.8059
Epoch 159/1000
237/237 [==============================] - 0s 56us/sample - loss: 0.4221 - accuracy: 0.8059
Epoch 160/1000
237/237 [==============================] - 0s 62us/sample - loss: 0.4219 - accuracy: 0.8059
Epoch 161/1000
237/237 [==============================] - 0s 60us/sample - loss: 0.4215 - accuracy: 0.8059
Epoch 162/1000
237/237 [==============================] - 0s 61us/sample - loss: 0.4213 - accuracy: 0.8059
Epoch 163/1000
237/237 [============================

237/237 [==============================] - 0s 67us/sample - loss: 0.4009 - accuracy: 0.8101
Epoch 231/1000
237/237 [==============================] - 0s 54us/sample - loss: 0.4005 - accuracy: 0.8101
Epoch 232/1000
237/237 [==============================] - 0s 52us/sample - loss: 0.4003 - accuracy: 0.8101
Epoch 233/1000
237/237 [==============================] - 0s 57us/sample - loss: 0.4002 - accuracy: 0.8101
Epoch 234/1000
237/237 [==============================] - 0s 56us/sample - loss: 0.4000 - accuracy: 0.8101
Epoch 235/1000
237/237 [==============================] - 0s 55us/sample - loss: 0.3999 - accuracy: 0.8101
Epoch 236/1000
237/237 [==============================] - 0s 55us/sample - loss: 0.3992 - accuracy: 0.8101
Epoch 237/1000
237/237 [==============================] - 0s 58us/sample - loss: 0.3990 - accuracy: 0.8101
Epoch 238/1000
237/237 [==============================] - 0s 52us/sample - loss: 0.3989 - accuracy: 0.8101
Epoch 239/1000
237/237 [============================

237/237 [==============================] - 0s 67us/sample - loss: 0.3841 - accuracy: 0.8101
Epoch 307/1000
237/237 [==============================] - 0s 58us/sample - loss: 0.3835 - accuracy: 0.8143
Epoch 308/1000
237/237 [==============================] - 0s 56us/sample - loss: 0.3831 - accuracy: 0.8143
Epoch 309/1000
237/237 [==============================] - 0s 60us/sample - loss: 0.3827 - accuracy: 0.8143
Epoch 310/1000
237/237 [==============================] - 0s 52us/sample - loss: 0.3828 - accuracy: 0.8143
Epoch 311/1000
237/237 [==============================] - 0s 50us/sample - loss: 0.3823 - accuracy: 0.8143
Epoch 312/1000
237/237 [==============================] - 0s 50us/sample - loss: 0.3821 - accuracy: 0.8101
Epoch 313/1000
237/237 [==============================] - 0s 50us/sample - loss: 0.3815 - accuracy: 0.8101
Epoch 314/1000
237/237 [==============================] - 0s 57us/sample - loss: 0.3815 - accuracy: 0.8143
Epoch 315/1000
237/237 [============================

...

238/238 [==============================] - 0s 61us/sample - loss: 0.3059 - accuracy: 0.8697
Epoch 643/1000
238/238 [==============================] - 0s 60us/sample - loss: 0.3061 - accuracy: 0.8697
Epoch 644/1000
238/238 [==============================] - 0s 69us/sample - loss: 0.3088 - accuracy: 0.8697
Epoch 645/1000
238/238 [==============================] - 0s 56us/sample - loss: 0.3090 - accuracy: 0.8697
Epoch 646/1000
238/238 [==============================] - 0s 53us/sample - loss: 0.3058 - accuracy: 0.8697
Epoch 647/1000
238/238 [==============================] - 0s 57us/sample - loss: 0.3057 - accuracy: 0.8697
Epoch 648/1000
238/238 [==============================] - 0s 61us/sample - loss: 0.3060 - accuracy: 0.8739
Epoch 649/1000
238/238 [==============================] - 0s 57us/sample - loss: 0.3059 - accuracy: 0.8739
Epoch 650/1000
238/238 [==============================] - 0s 57us/sample - loss: 0.3057 - accuracy: 0.8697
Epoch 651/1000
238/238 [============================

238/238 [==============================] - 0s 50us/sample - loss: 0.3026 - accuracy: 0.8697
Epoch 719/1000
238/238 [==============================] - 0s 47us/sample - loss: 0.3022 - accuracy: 0.8697
Epoch 720/1000
238/238 [==============================] - 0s 52us/sample - loss: 0.3022 - accuracy: 0.8697
Epoch 721/1000
238/238 [==============================] - 0s 54us/sample - loss: 0.3026 - accuracy: 0.8782
Epoch 722/1000
238/238 [==============================] - 0s 52us/sample - loss: 0.3031 - accuracy: 0.8782
Epoch 723/1000
238/238 [==============================] - 0s 52us/sample - loss: 0.3031 - accuracy: 0.8739
Epoch 724/1000
238/238 [==============================] - 0s 50us/sample - loss: 0.3026 - accuracy: 0.8782
Epoch 725/1000
238/238 [==============================] - 0s 52us/sample - loss: 0.3022 - accuracy: 0.8782
Epoch 726/1000
238/238 [==============================] - 0s 54us/sample - loss: 0.3024 - accuracy: 0.8697
Epoch 727/1000
238/238 [============================

238/238 [==============================] - 0s 69us/sample - loss: 0.2993 - accuracy: 0.8739
Epoch 795/1000
238/238 [==============================] - 0s 75us/sample - loss: 0.2995 - accuracy: 0.8739
Epoch 796/1000
238/238 [==============================] - 0s 70us/sample - loss: 0.3022 - accuracy: 0.8655
Epoch 797/1000
238/238 [==============================] - 0s 68us/sample - loss: 0.2990 - accuracy: 0.8697
Epoch 798/1000
238/238 [==============================] - 0s 70us/sample - loss: 0.2996 - accuracy: 0.8782
Epoch 799/1000
238/238 [==============================] - 0s 67us/sample - loss: 0.3009 - accuracy: 0.8739
Epoch 800/1000
238/238 [==============================] - 0s 65us/sample - loss: 0.3002 - accuracy: 0.8739
Epoch 801/1000
238/238 [==============================] - 0s 69us/sample - loss: 0.2991 - accuracy: 0.8782
Epoch 802/1000
238/238 [==============================] - 0s 67us/sample - loss: 0.3004 - accuracy: 0.8697
Epoch 803/1000
238/238 [============================

238/238 [==============================] - 0s 60us/sample - loss: 0.2970 - accuracy: 0.8824
Epoch 871/1000
238/238 [==============================] - 0s 59us/sample - loss: 0.2970 - accuracy: 0.8782
Epoch 872/1000
238/238 [==============================] - 0s 61us/sample - loss: 0.2983 - accuracy: 0.8782
Epoch 873/1000
238/238 [==============================] - 0s 58us/sample - loss: 0.2975 - accuracy: 0.8824
Epoch 874/1000
238/238 [==============================] - 0s 59us/sample - loss: 0.2964 - accuracy: 0.8824
Epoch 875/1000
238/238 [==============================] - 0s 63us/sample - loss: 0.2968 - accuracy: 0.8782
Epoch 876/1000
238/238 [==============================] - 0s 59us/sample - loss: 0.2972 - accuracy: 0.8739
Epoch 877/1000
238/238 [==============================] - 0s 58us/sample - loss: 0.2962 - accuracy: 0.8782
Epoch 878/1000
238/238 [==============================] - 0s 58us/sample - loss: 0.2964 - accuracy: 0.8824
Epoch 879/1000
238/238 [============================

238/238 [==============================] - 0s 57us/sample - loss: 0.2951 - accuracy: 0.8824
Epoch 947/1000
238/238 [==============================] - 0s 59us/sample - loss: 0.2945 - accuracy: 0.8866
Epoch 948/1000
238/238 [==============================] - 0s 61us/sample - loss: 0.2947 - accuracy: 0.8866
Epoch 949/1000
238/238 [==============================] - 0s 57us/sample - loss: 0.2938 - accuracy: 0.8866
Epoch 950/1000
238/238 [==============================] - 0s 58us/sample - loss: 0.2935 - accuracy: 0.8866
Epoch 951/1000
238/238 [==============================] - 0s 56us/sample - loss: 0.2944 - accuracy: 0.8866
Epoch 952/1000
238/238 [==============================] - 0s 52us/sample - loss: 0.2945 - accuracy: 0.8866
Epoch 953/1000
238/238 [==============================] - 0s 53us/sample - loss: 0.2936 - accuracy: 0.8866
Epoch 954/1000
238/238 [==============================] - 0s 56us/sample - loss: 0.2938 - accuracy: 0.8866
Epoch 955/1000
238/238 [============================

In [ ]:
#10-fold
[[0.3194412589073181, 0.8333333],
 [0.34246203303337097, 0.8666667],
 [0.35849910974502563, 0.8],
 [0.3818192481994629, 0.8666667],
 [0.6909805536270142, 0.53333336],
 [0.41834938526153564, 0.76666665],
 [0.5079019069671631, 0.8333333],
 [0.292437344789505, 0.8965517],
 [0.6209121942520142, 0.7241379],
 [0.39089736342430115, 0.82758623]]
 79%
#5-fold
[[0.3283263782660166, 0.8833333],
 [0.5756371696790059, 0.78333336],
 [0.4828665559574709, 0.7966102],
 [0.5146383481510615, 0.779661],
 [0.6075216461036165, 0.779661]]`
 80%

In [13]:
from tensorflow.keras import models
m=models.load_model("best_nn.hdf5")
m.summary()
m.evaluate(X_test,y_test,verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 60        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
59/1 - 0s - loss: 0.2665 - accuracy: 0.8644


[0.26766378798727264, 0.86440676]

In [ ]:
################################## data preprocessing
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

from sklearn.preprocessing import StandardScaler as ss
sc = ss()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#########################################   SVM   #############################################################
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf')
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print('Accuracy for training set for svm = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for svm = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
#########################################   Naive Bayes  #############################################################
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)


# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print('Accuracy for training set for Naive Bayes = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Naive Bayes = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
#########################################   Decision Tree  #############################################################
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print('Accuracy for training set for Decision Tree = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Decision Tree = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
#########################################  Random Forest  #############################################################
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print('Accuracy for training set for Random Forest = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Random Forest = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
###############################################################################
# applying lightGBM
import lightgbm as lgb

d_train = lgb.Dataset(X_train, label = y_train)
params = {}

clf = lgb.train(params, d_train, 100)
#Prediction
y_pred = clf.predict(X_test)
#convert into binary values
for i in range(0, len(y_pred)):
    if y_pred[i]>= 0.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0
       
from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = clf.predict(X_train)

for i in range(0, len(y_pred_train)):
    if y_pred_train[i]>= 0.5:       # setting threshold to .5
       y_pred_train[i]=1
    else:  
       y_pred_train[i]=0
       
cm_train = confusion_matrix(y_pred_train, y_train)
print()
print('Accuracy for training set for LightGBM = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for LightGBM = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
###############################################################################
# applying XGBoost

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.20, random_state = 0)

from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = xg.predict(X_train)

for i in range(0, len(y_pred_train)):
    if y_pred_train[i]>= 0.5:       # setting threshold to .5
       y_pred_train[i]=1
    else:  
       y_pred_train[i]=0
       
cm_train = confusion_matrix(y_pred_train, y_train)
print()
print('Accuracy for training set for XGBoost = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for XGBoost = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))